In [1]:
import cv2 as cv
import re
import os
import psycopg2
from scipy.misc import bytescale
import numpy as np

In [2]:
def writeContours(src, dest, name):
    
    try: 
        os.makedirs(dest)
    except OSError:
        if not os.path.isdir(dest):
            raise

    files = os.listdir(src);
    frames = sum(1 for file in files if file.endswith(".tif"))

    for i in range(frames):
        image = cv.imread(src + name + "{:03.0f}.tif".format(i))

        imgray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        contours, hierarchy = cv.findContours(imgray, cv.RETR_EXTERNAL ,  cv.CHAIN_APPROX_NONE)
       
        os.mkdir(dest + str(i))
        for j in  range(len(contours)):
            if len(contours[j]) > 10:
                f= open(dest + str(i)+"\\"+str(j)+".wkt","w+")
                line = re.sub("[\[\]]", "", str(contours[j]))
                line = line.replace("\n\n", ",")
                f.write("POLYGON(("+line.strip()+"))\n")
                f.close()

In [3]:
src = "..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\02_GT\\TRA\\segmented\\0\\";
dest = src + "man_track\\";
name = "man_track";


writeContours(src,dest,name)

WindowsError: [Error 183] Impossível criar um ficheiro quando esse ficheiro já existe: '..\\Datasets\\TrackingChallenge\\Fluo-N2DH-SIM+\\02_GT\\TRA\\segmented\\0\\man_track\\0'

In [14]:
def LoadVideoFrames(pHost, pDatabase, pUser, pPwd,path):
    conn = None
    try:
   
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=pHost,port=5434,database=pDatabase, user=pUser, password=pPwd)
     
        # create a cursor
        cur = conn.cursor()
       
        # execute a statement
        print('PostgreSQL connected')
   
        #cur.execute("delete from mytable where name = '" + pObjectName + "'");
   
        allLines = []
        time = 0
       
        dirlist = os.listdir(path)
        for dir in dirlist:
            frameN = int(''.join(filter(str.isdigit, dir)));
            fileList = os.listdir(path+dir)
            for file in fileList:
                time = file.replace('.wkt', '').replace('interpolation', '')
            
                with open(path + dir+'/' + file) as f:
                    for line in f:
                        print line
                        print(path + dir+'/' + file)
                        lineTmp = line.split(',');
                        pol = line[:-3]+','+lineTmp[0][9:]+line[-3:];
                        if (line.startswith('POLYGON')):
                            command = "insert into lifetime values (%d,%d,ST_RotateX('%s',pi()))" % (int(time),frameN,pol);
                            #print (command)
                            cur.execute(command);
                        else:
                            print(time);

                   
         
        # close the communication with the PostgreSQL
        cur.close()
        conn.commit()
   
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
       
            conn.close()
            print('Database connection closed.')

In [16]:
host = "localhost";
database = "PhC-C2DL-PSC_ST";
user = "postgres";
pw = "postgres";
path ="..\\Datasets\\TrackingChallenge\\PhC-C2DL-PSC\\01_ST\\SEG\\segmented\\0\\man_track\\";



LoadVideoFrames(host,database,user,pw,path)

Connecting to the PostgreSQL database...
PostgreSQL connected
POLYGON((890 734, 889 735, 888 736, 887 737, 886 738, 885 738, 884 738, 883 739, 882 739, 881 739, 880 740, 879 740, 878 740, 877 740, 876 741, 875 741, 874 742, 873 743, 872 743, 871 743, 870 743, 869 743, 868 743, 867 743, 866 744, 865 745, 864 746, 863 747, 862 747, 861 748, 860 749, 859 750, 858 750, 857 751, 856 751, 855 752, 854 752, 853 753, 852 753, 851 753, 850 754, 849 755, 848 756, 847 756, 846 756, 845 757, 844 757, 843 758, 842 758, 841 758, 840 758, 839 759, 838 759, 837 760, 836 761, 835 762, 834 763, 833 764, 832 764, 832 765, 831 766, 830 767, 829 768, 828 768, 827 769, 826 770, 825 771, 824 772, 823 772, 822 773, 821 774, 820 775, 819 776, 818 777, 817 778, 816 778, 815 779, 814 779, 813 780, 812 781, 811 781, 810 782, 809 783, 808 784, 807 784, 806 785, 805 786, 804 787, 803 787, 802 788, 801 788, 800 788, 799 788, 798 788, 797 789, 796 789, 795 789, 794 790, 793 790, 792 790, 791 791, 790 791, 789 791, 78

In [17]:
import psycopg2
import os
def LoadCells(pHost, pDatabase, pUser, pPwd, pObjectName):
    conn = None
    try:
   
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(host=pHost,port=5434,database=pDatabase, user=pUser, password=pPwd)
     
        # create a cursor
        cur = conn.cursor()
       
        # execute a statement
        print('PostgreSQL connected')
   
        #cur.execute("delete from mytable where name = '" + pObjectName + "'");

       
       
        with open('..\\Datasets\\TrackingChallenge\\Fluo-N2DL-HeLa\\01_GT\\TRA\\man_track.txt') as f:
                for line in f:
                    values = line.split()
                    print(values[0])
                    command = 'insert into cells VALUES (%d,%d,%d,%d)' % (int(values[0]),int(values[1]),int(values[2]),int(values[3]));
                    print (command)
                    cur.execute(command);
                   
         
        # close the communication with the PostgreSQL
        cur.close()
        conn.commit()
   
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
       
            conn.close()
            print('Database connection closed.')
 
LoadCells("localhost","Fluo-N2DL-HeLa_ST","postgres","postgres", "cells")

Connecting to the PostgreSQL database...
PostgreSQL connected
1
insert into cells VALUES (1,0,30,0)
8
insert into cells VALUES (8,31,91,1)
9
insert into cells VALUES (9,31,91,1)
10
insert into cells VALUES (10,0,4,0)
12
insert into cells VALUES (12,0,38,0)
14
insert into cells VALUES (14,39,91,12)
15
insert into cells VALUES (15,39,91,12)
16
insert into cells VALUES (16,0,22,0)
18
insert into cells VALUES (18,23,91,16)
19
insert into cells VALUES (19,23,91,16)
20
insert into cells VALUES (20,43,91,0)
21
insert into cells VALUES (21,63,91,0)
22
insert into cells VALUES (22,0,1,0)
23
insert into cells VALUES (23,3,3,0)
24
insert into cells VALUES (24,3,37,0)
25
insert into cells VALUES (25,0,3,0)
28
insert into cells VALUES (28,4,49,25)
29
insert into cells VALUES (29,4,48,25)
30
insert into cells VALUES (30,0,1,0)
32
insert into cells VALUES (32,2,39,30)
33
insert into cells VALUES (33,2,44,30)
35
insert into cells VALUES (35,91,91,0)
36
insert into cells VALUES (36,40,91,32)
37
insert 

In [ ]:
a = "adsafas"

In [ ]:
print a[:5]